In [ ]:
import networkx as nx
import numpy as np
import scipy
from scipy import stats
from statsmodels.stats import weightstats as w
import powerlaw

#1st Question
#i
def random_networks_generator(n,p,num_networks=1, directed=False, seed=123):
    result_list=[]
    for i in range(num_networks):    
        seed=seed+i    
        g=nx.erdos_renyi_graph(n, p, seed, directed)    
        result_list.append(g)
    return result_list

#ii
def network_stats(network):
    #degree_list
    degrees = [val for (node, val) in network.degree()]
    
    #Average degrees distribution
    k=(sum(degrees)/network.number_of_nodes())
    result_list=[]
    result_list.append(("degrees_avg", k))
    
    #Standard deviation degrees distribution 
    std=np.std(degrees)
    result_list.append(("degrees_std", std))
      
    #degrees_min
    min_degree=min(degrees)
    result_list.append(("degrees_min", min_degree))
    
    #degrees_max
    max_degree=max(degrees)
    result_list.append(("degrees_max", max_degree))
    
    #average shortest path length
    spl=nx.average_shortest_path_length(network)
    result_list.append(("spl", spl))
    
    #The network’s diameter
    diameter=nx.diameter(network)
    result_list.append(("diameter", diameter))
    
    return dict(result_list)

#iii    
def networks_avg_stats(networks_list):
    
    result_list=[]
    average_sum=0
    min_sum=0
    max_sum=0
    spl_sum=0
    std_sum=0
    diam_sum=0
    
    for network in networks_list:
        single_network_dict=network_stats(network)
        average_sum = average_sum+single_network_dict["degrees_avg"]
        min_sum = min_sum+single_network_dict["degrees_min"]
        max_sum = max_sum+single_network_dict["degrees_max"]
        std_sum=std_sum+single_network_dict["degrees_std"]
        spl_sum = spl_sum+single_network_dict["spl"]
        diam_sum = diam_sum+single_network_dict["diameter"]
        
    result_list=[("degrees_avg", average_sum/len(networks_list)),("degrees_std", std_sum/len(networks_list)),("degrees_min", min_sum/len(networks_list)),("degrees_max", max_sum/len(networks_list)), ("spl", spl_sum/len(networks_list)),("diameter", diam_sum/len(networks_list))]
    return dict(result_list)     


#2nd Question
#i
    randomslst=nx.read_gpickle("rand_nets.p")

#ii
def rand_net_hypothesis_testing(network, theoretical_p, alpha=0.05):
    #degree_list
    degrees = [val for (node, val) in network.degree()]
    
    #number_of_nodes
    n= network.number_of_nodes()
       
    #As taught in class <k>=(n-1)*p
    theoretical_k=(n-1)*theoretical_p
    
    z, pvalue=w.ztest(degrees, x2=None, value=theoretical_k)
    
    if pvalue<alpha:
        return (pvalue,"reject")
    else:
        return(pvalue,"accept")
        
#iii       
def most_probable_p(network):

    plst=[0.01,0.1,0.3,0.6]
    reslst=[]
    for i in range(len(plst)):
        reslst.append(rand_net_hypothesis_testing(network, plst[i], alpha=0.05))
        
    for restuple in reslst:
        if restuple[1]=="accept":
            result=plst[reslst.index(restuple)]
        else:
            result=-1

    return result
        

#3rd Question - Scale free networks
#i
    scalefreelst=nx.read_gpickle("scalefree_nets.p")
#ii
   
def find_opt_gamma(network,treat_as_social_network=True):
    #degree_list
    degrees = [val for (node, val) in network.degree()]

    fit=powerlaw.Fit((degrees), discrete=treat_as_social_network)
   
    return fit.power_law.alpha; 

#4th Question 
#i
    mixedlst=nx.read_gpickle("mixed_nets.p")  
#ii 
def network_classifier(network):
    #degree_list
    degrees = [val for (node, val) in network.degree()]

    fit=powerlaw.Fit((degrees))
    
    alpha= fit.power_law.alpha
    
    kmax=max(degrees)
    
    n= network.number_of_nodes()
    
    if 2 < alpha < 3:
        result=2
    
    elif alpha > 3:
        result= 1
    
    else:
        scale_kmax_approx=n^(1/(alpha-1))
        random_kmax_approx=np.log(n)
        if abs(scale_kmax_approx-kmax)>abs(random_kmax_approx-kmax):
            result=1
        else:
            result=2
    return result

